In [1]:
import json
import pandas as pd
import requests
from sqlalchemy import create_engine
import sqlite3 as db
from sqlite3 import Error
from datetime import datetime

%run C:\Users\Joshua\Jupyter_Notebook_Folders\APIkeys.py
%run C:\Users\Joshua\Jupyter_Notebook_Folders\MacroData2\db_functions.py

In [2]:
def get_BLS_data(seriesIDs, startyear, endyear):
    """
    get data from BLS
    """
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'  #this will not change
    headers = {'Content-type': 'application/json'}  #This will not changed !

    # For the key seriesid enter a list of series names you wish to download
    # For the key startyear enter the start year inside ""
    # For the key endyear enter the end year inside ""
    
    parameters = { 
        "seriesid":seriesIDs,
        "startyear":str(startyear), 
        "endyear":str(endyear),
        "catalog":True, 
        "calculations":False, 
        "annualaverage":False,
        "aspects":False,
        "registrationkey":os.environ['BLS_API_key'] 
     }

    data = json.dumps(parameters) # Converts the Python dictionary to JSON

    p = requests.post(base_url, data=data, headers=headers)
    json_data = json.loads(p.text)
    
    message = ""
    if json_data['message']:
        #message = "For series " + seriesIDs + ", no data for years: "
        for i in range(len(json_data['message'])):
            message += json_data['message'][i][-4:] + ", "
    
    return message, json_data 

In [3]:
message, json_data = get_BLS_data(["CUSR0000SA0"], 2005, 2024)

In [4]:
json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 222,
 'message': [],
 'Results': {'series': [{'seriesID': 'CUSR0000SA0',
    'catalog': {'series_title': 'All items in U.S. city average, all urban consumers, seasonally adjusted',
     'series_id': 'CUSR0000SA0',
     'seasonality': 'Seasonally Adjusted',
     'survey_name': 'Consumer Price Index for All Urban Consumers (CPI-U)',
     'survey_abbreviation': 'CU',
     'measure_data_type': 'All items',
     'area': 'U.S. city average',
     'item': 'All items'},
    'data': [{'year': '2024',
      'period': 'M07',
      'periodName': 'July',
      'latest': 'true',
      'value': '313.534',
      'footnotes': [{}]},
     {'year': '2024',
      'period': 'M06',
      'periodName': 'June',
      'value': '313.049',
      'footnotes': [{}]},
     {'year': '2024',
      'period': 'M05',
      'periodName': 'May',
      'value': '313.225',
      'footnotes': [{}]},
     {'year': '2024',
      'period': 'M04',
      'periodName': 'April',
    

In [5]:
def create_data_dataframe(json_data):
    df = pd.DataFrame(json_data['Results']['series'][0]['data'])
    df['month'] = df['year'] + df['period'].str.replace('M','')
    df['month'] = pd.to_datetime(df['month'],format='%Y%m')
    df['month'] = df['month'].dt.date.apply(lambda x: x.strftime('%Y-%m'))
    df = df.sort_values(by=['month'])
    df['pct_changeMoM'] = df['value'].astype(float).pct_change() * 100
    df['pct_changeYoY'] = df['value'].astype(float).pct_change(12) * 100
    df['pct_changeMoM_Ann'] = ((df['value'].astype(float).pct_change() + 1) ** 12 - 1) * 100
    df = df.drop(columns=['year', 'period','periodName', 'latest','footnotes'])
    
    return df

In [6]:
df = create_data_dataframe(json_data)
df

,value,month,pct_changeMoM,pct_changeYoY,pct_changeMoM_Ann
234,191.6,2005-01,NaN,NaN,NaN
233,192.4,2005-02,0.417537,NaN,5.127117
232,193.1,2005-03,0.363825,NaN,4.454336
231,193.7,2005-04,0.310720,NaN,3.793024
230,193.6,2005-05,-0.051626,NaN,-0.617759
...,...,...,...,...,...
4,312.230,2024-03,0.378069,3.475131,4.632370
3,313.207,2024-04,0.312910,3.357731,3.820226
2,313.225,2024-05,0.005747,3.250210,0.068986
1,313.049,2024-06,-0.056190,2.975629,-0.672196


In [80]:
df.to_csv('CPI.csv', index=False)

In [8]:
def create_dict_dataframe(json_data):
    df_dict = pd.DataFrame()
    df_dict['series_title'] = json_data['Results']['series'][0]['catalog']['series_title'],
    df_dict['series_id'] = json_data['Results']['series'][0]['catalog']['series_id'],
    df_dict['seasonality'] = json_data['Results']['series'][0]['catalog']['seasonality'],
    df_dict['survey_long_name'] = json_data['Results']['series'][0]['catalog']['survey_name'],
    df_dict['survey_short_name'] = json_data['Results']['series'][0]['catalog']['survey_name'][-6:-1],
    df_dict['survey_abbreviation'] = json_data['Results']['series'][0]['catalog']['survey_abbreviation'],
    df_dict['measure_data_type'] = json_data['Results']['series'][0]['catalog']['measure_data_type'],
    df_dict['area'] = json_data['Results']['series'][0]['catalog']['area'],
    df_dict['item'] = json_data['Results']['series'][0]['catalog']['item']
    df_dict = df_dict.set_index(df_dict['series_id'])
    df_dict = df_dict.drop(columns=['series_id'])
    
    return df_dict

In [9]:
df_dict = create_dict_dataframe(json_data)
df_dict

,series_title,seasonality,survey_long_name,survey_short_name,survey_abbreviation,measure_data_type,area,item
series_id,,,,,,,,
CUSR0000SA0,"All items in U.S. city average, all urban cons...",Seasonally Adjusted,Consumer Price Index for All Urban Consumers (...,CPI-U,CU,All items,U.S. city average,All items


In [12]:
def list_db_tables(db_name):
    Conn = db.connect(db_name)
    sql_query = """SELECT name FROM sqlite_master  
      WHERE type='table';"""
    cursor = Conn.cursor()
    cursor.execute(sql_query)
    print(cursor.fetchall())
    Conn.close() 

In [23]:
df_dict.index[0] + '_data'

'CUSR0000SA0_data'

In [28]:
def create_data_table(tbl_name, db_name):
    """
    create data table to hold series values
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code1 = ('''CREATE TABLE IF NOT EXISTS ''' + tbl_name + ''' (
                    month 
                    ,value
                    ,pct_changeMoM
                    ,pct_changeYoY
                    ,pct_changeMoM_Ann
                );''')
    
    c.execute(sql_code1) 
    
    # Save (commit) the changes
    conn.commit()
    
    conn.close()

In [29]:
#create_data_table(df_dict.index[0] + '_data', 'MacroData.db')

In [13]:
list_db_tables('MacroData.db')

[('CUSR0000SA0_data',)]


In [10]:
conn = db.connect('MacroData.db')
df.to_sql(df_dict.index[0] + '_data', conn, if_exists='replace')

235